In [1]:
from __future__ import print_function

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard,ReduceLROnPlateau

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
# from tensorflow.keras.datasets import mnist, cifar10
import tensorflow_addons as tfa

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.io import loadmat 
import mat73
from datetime import datetime
import ipynbname

print("Packages Loaded")

Packages Loaded


In [2]:
# wandb login

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="my-test-project", entity="ibksolar", name="NewDataset_1stTry")


wandb: Currently logged in as: ibksolar (use `wandb login --relogin` to force relogin)


In [3]:
# base_path = '..\\..\\all_block_data\Dec_Train_block_len_21_011121_2331'
base_path = '../all_block_data/Old_data/Dec_Train_block_len_21_231121_1531'

# Confirm path is right...
print(f'{os.path.isdir(base_path)}')

True


In [6]:

raw_data1 = mat73.loadmat(base_path + '/echo_cnn_in_out_jstars.mat')
all_data = raw_data1['echo_cnn_input']
all_target = raw_data1['echo_cnn_target']
coords = raw_data1['coords']
echo_idx = raw_data1['orig_echo_idx']

# Set all nan in the data to zero
nan_idx = np.isnan(all_data).any(axis =-1)
all_target[nan_idx] = 0
all_data[ np.isnan(all_data) ]= 0


In [7]:
# Truncate data because data after truncate point is notgood for training
echo_idx = np.asarray(echo_idx)
stop_val = 600

stop_list, = np.where(echo_idx == stop_val)
stop_idx = stop_list[-1]

all_data = all_data[:stop_idx]
all_target = all_target[:stop_idx]

print(f'Data shape {all_data.shape}')
print(f'Target shape {all_target.shape}')


Data shape (917311, 315)
Target shape (917311,)


In [8]:
row_length = 21 # CHANGE HERE <==
col_length = 15

# Check that the dimension of data is correct
if all_data.shape[1] == row_length*col_length:
    print('Dimensions match')
else:
    print(f' Row block length:{row_length} and col length:{col_length} does not match Data dimension:{all_data.shape[1]}') 
        

Dimensions match


In [ ]:
max_class = row_length 

# Highest class is mapped to row_length+1
all_target[all_target == max_class+1 ] = 0


shuffle = 0
if shuffle:
    random.Random(1337).shuffle(all_data)
    random.Random(1337).shuffle(all_target)
    # random.Random(1337).shuffle(all_idx)

## Prep data
train_size = int(np.floor(0.8*len(all_target)));
test_size = int(np.round( 0.15* all_data.shape[0] ))
val_size = all_data.shape[0] -train_size - test_size


x_train = all_data[0:train_size,:]
# x_train = np.reshape( x_train, (x_train.shape[0],max_class,-1,1) )

x_test = all_data[train_size:train_size+test_size,:]
# x_test = np.reshape( x_test, (x_test.shape[0],max_class,-1,1) )

x_val = all_data[-val_size:,:]
# x_val = np.reshape( x_val,(x_val.shape[0],max_class,-1,1) )

y_train = all_target[:train_size]
y_test  = all_target[train_size:train_size+test_size]
y_val = all_target[-val_size:]

var_input_shape = x_train.shape[1:] # 240 columns
num_classes = max_class+1 # layers

# Convert labels to categorical orthonormal vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)


print(f'Shape of X_train:{x_train.shape}  X_test:{x_test.shape}')
print(f'Shape of y_train:{y_train.shape}  y_test:{y_test.shape}')

Shape of X_train:(733848, 315)  X_test:(137597, 315)
Shape of y_train:(733848, 22)  y_test:(137597, 22)


In [ ]:
# Create your own ResNet here.

# Skip 3 connection version of base model
drop_out = 0.3
nodes = 256
shp = x_train.shape[1]

inputs = tf.keras.Input(shape=(shp,), name='img')
# aug_inputs = data_augmentation(inputs)

x = Dense(nodes, activation='relu')(inputs)
block_1_output = Dense(nodes, activation='relu')(x)

x = Dense(nodes)(block_1_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)

x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)

x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_2_output = tf.keras.layers.add([x, block_1_output])

x = Dense(nodes)(block_2_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_3_output = tf.keras.layers.add([x, block_2_output])

x = Dense(nodes)(block_3_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_4_output = tf.keras.layers.add([x, block_3_output])

x = Dense(nodes)(block_4_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_5_output = tf.keras.layers.add([x, block_4_output])


x = Dense(nodes)(block_5_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_6_output = tf.keras.layers.add([x, block_5_output])

x = Dense(nodes)(block_6_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_7_output = tf.keras.layers.add([x, block_6_output])

x = Dense(nodes)(block_7_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_8_output = tf.keras.layers.add([x, block_7_output])

x = Dense(nodes)(block_8_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_9_output = tf.keras.layers.add([x, block_8_output])

x = Dense(nodes)(block_9_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(nodes)(x)
x = BatchNormalization()(x)

block_10_output = tf.keras.layers.add([x, block_9_output])


x = Dense(nodes, activation='relu')(block_10_output)
x = Dropout(drop_out)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name='Plain_Old_Resnet')

top_K = 3
weight_decay = 0.3

optimizer = tfa.optimizers.AdamW(learning_rate=1e-1, weight_decay=weight_decay) #Adam(amsgrad=True)

model.compile( Adam(learning_rate=1e-4,amsgrad=True), loss= tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy',tf.keras.metrics.TopKCategoricalAccuracy(top_K, name="top-3-accuracy")])  #label_smoothing=0.05, tf.keras.losses.KLDivergence()

time_stamp = datetime.strftime( datetime.now(),'%d_%B_%y_%H%M')
earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('jstars_june_{epoch:02d}_{val_loss:.2f}.hdf5', save_best_only=True, monitor='val_loss', mode='min')

# mcp_path = f'{base_path}_/tmp/{time_stamp}_jstars_Epoch_{epoch:02d}_ValLoss{val_loss:.2f}.hdf5'
# mcp_save = ModelCheckpoint('/tmp/jstars_Epoch_{epoch:02d}_ValLoss{val_loss:.2f}.hdf5', save_best_only=True, monitor='val_loss', mode='min') #{base_path}_

reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=0, emin_delta =1e-4, mode='min')

#TO DO: Would normalizing with from sklearn.preprocessing import MinMaxScaler help? Try later
model.fit(x_train, y_train,
          epochs= 50, 
          batch_size= 64, 
          validation_data=(x_test, y_test)) #mcp_save, callbacks=[reduce_lr_loss]



Epoch 1/50
 5812/11467 [==============>...............] - ETA: 22:04 - loss: 2.2102 - accuracy: 0.4002 - top-3-accuracy: 0.7758

In [8]:
# Define Sequential model with 3 layers
num_epochs = 100
batch_size = 64
nodes = 256
learning_rate = 1e-4

wandb.config = {
  "learning_rate": "learning_rate",
  "epochs": num_epochs,
  "batch_size": batch_size,
  "nodes": nodes,
  "row_length":row_length,
  "base_path":base_path

}

input_shape = x_train.shape[1:] #(21,15,1)

def ResNetBlock(x):
    x =   layers.Conv2D(filters=64, kernel_size=3, padding="same")(x) #input_layer
    conv1 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(x) # input_layer, Conv1D
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.ReLU()(conv1)

    conv2 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.ReLU()(conv2)

    conv3 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.ReLU()(conv3)
    
    
    conv3 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.ReLU()(conv3)
    
    
    conv3 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.ReLU()(conv3)
    
    conv3 = layers.add([x,conv3])
    x = layers.ReLU()(conv3) # Overwrite x
    
    return x


def make_model(input_shape):
    input_layer = layers.Input(input_shape)
    
    x =   layers.Conv2D(filters=64, kernel_size=3, padding="same")(input_layer)
    
    for _ in range(20):
        x = ResNetBlock(x)
    
    # conv1 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(input_layer) #Conv1D
    # conv1 = layers.BatchNormalization()(conv1)
    # conv1 = layers.ReLU()(conv1)
    # conv2 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(conv1)
    # conv2 = layers.BatchNormalization()(conv2)
    # conv2 = layers.ReLU()(conv2)
    # conv3 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(conv2)
    # conv3 = layers.BatchNormalization()(conv3)
    # conv3 = layers.ReLU()(conv3)
    
    gap = layers.GlobalAveragePooling2D()(x)
    gap = layers.Flatten()(gap)

    output_layer = layers.Dense(num_classes, activation="softmax")(gap)

    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

model = make_model(input_shape = input_shape) #x_train.shape[1:]


callbacks = [
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.25, patience=5, min_lr=0.0001),
    EarlyStopping(monitor="val_loss", patience=25, verbose=1), 
    WandbCallback()
]


# Poly Rate scheduler
starter_learning_rate = 0.1
end_learning_rate = 0.001
decay_steps = 1000
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    starter_learning_rate,
    decay_steps,
    end_learning_rate,
    power=0.5)

# opt1 = tf.keras.optimizers.RMSprop(learning_rate=0.1,rho=0.9,momentum=0.9, epsilon=1e-07,centered=True,name="RMSprop")
opt2 = tf.keras.optimizers.Adam(learning_rate=learning_rate,amsgrad=True)
poly_rate = tf.keras.optimizers.SGD(learning_rate = learning_rate_fn)
poly_rate2 = tf.keras.optimizers.Adam(learning_rate = learning_rate_fn)

# top_K = 3
# model.compile( optimizer = opt, loss= 'categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.TopKCategoricalAccuracy(top_K, name="top-3-accuracy")])  #label_smoothing=0.05, tf.keras.losses.KLDivergence()

model.compile( optimizer=poly_rate2,loss="categorical_crossentropy", metrics=["accuracy"],) #sparse_categorical_accuracy


history = model.fit(x_train, y_train,
          epochs= num_epochs, 
          batch_size= batch_size, 
          validation_data=(x_test, y_test),
         callbacks=callbacks) #mcp_save, callbacks=[reduce_lr_loss]



Epoch 1/100
15663/15663 [==============================] - 8270s 527ms/step - loss: 1558866298929152.0000 - accuracy: 0.1153 - val_loss: 2.6399 - val_accuracy: 0.0999


C:\Users\i368o351\Anaconda3\envs\ibk_sept21_tf2.5\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
15663/15663 [==============================] - 8170s 522ms/step - loss: 2.5868 - accuracy: 0.1152 - val_loss: 2.6337 - val_accuracy: 0.0999
Epoch 3/100
15663/15663 [==============================] - 8128s 519ms/step - loss: 2.5868 - accuracy: 0.1156 - val_loss: 2.6303 - val_accuracy: 0.0999
Epoch 4/100
15663/15663 [==============================] - 8161s 521ms/step - loss: 2.5868 - accuracy: 0.1155 - val_loss: 2.6271 - val_accuracy: 0.0999
Epoch 5/100
15663/15663 [==============================] - 8302s 530ms/step - loss: 2.5868 - accuracy: 0.1154 - val_loss: 2.6247 - val_accuracy: 0.0999
Epoch 6/100
15663/15663 [==============================] - 8239s 526ms/step - loss: 2.5868 - accuracy: 0.1155 - val_loss: 2.6225 - val_accuracy: 0.0999
Epoch 7/100
10128/15663 [==================>...........] - ETA: 45:46 - loss: 2.5872 - accuracy: 0.1154

InternalError: Could not synchronize CUDA stream: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered

In [ ]:
_, accuracy = model.evaluate(x_val, y_val)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
# print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

_,new_col_length,new_row_length = x_train.shape 
model.save(f'{base_path}//{ipynbname.name()}//{time_stamp}_Acc_{accuracy:.3f}_{new_row_length}x{new_col_length}.h5')